In [111]:
import pymysql.cursors
import pandas as pd
import numpy as np
import re
import math

In [112]:
def connect_to_database(db="UltimateGuitarTabs"):
    """
    Connects to MySQL database and 
    returns the connection"""
    db = pymysql.connect(host="localhost",  # your host 
                         user="root",       # username
                         passwd="",     # password
                         db=db)   # name of the database
    return db

def print_table(tableName):
    """
    Prints database table
    """
    db = connect_to_database()
    cur = db.cursor()

    # Select data from table using SQL query.
    cur.execute("SELECT * FROM "+tableName)
    # print the first and second columns      
    for row in cur.fetchall() :
        print(row)
        
def get_table(tableName):
    db = connect_to_database()
    cur = db.cursor()

    # Select data from table using SQL query.
    cur.execute("SELECT * FROM "+tableName)
    
    return(cur.fetchall())


In [119]:
Chords = pd.DataFrame(list(get_table('Chords')), columns=['id','Song', 'Artist', 'Key', 'Capo', 'Chords'])
Chords

,id,Song,Artist,Key,Capo,Chords
0,23,Beer Drinkers And Hell Raisers,ZZ Top,,0,"Em7,E,D,A5,A6,B5,B6,Em7,Em7,E,D,A5,A6,A5,A6,Em..."
1,73,Harvest Moon,Neil Young,,0,"G,C,C,G,C,G,C,C,G,C,G,C,C,G,C,G,C,G,G,Bm,C,Am,..."
2,74,Unplugged,Neil Young,,0,"G,C,G,G,C,G,C,G,C,G,D,C,G,G,C,G,G,C,G,C,C,Am,C..."
3,105,Ball And Chain,XTC,,0,"C,C7sus4,C,C7sus4,C,C7sus4,Bb,F,C,G5,D,F,C,G5,..."
4,118,Los Angeles,X,,0,"E,C,D,G,A,A,G,D,C,A,G,D,C,A,G,C,A,G,C,A,G,C,A,..."
5,120,The Hungry Wolf,X,,0,"C,B,B,C,B,C,B,D,E,C,B,C,B,C,B,D,E,E,C,E,C,B,C,..."
6,234,Roses Are Free,Ween,,0,"F,G,G#,A,A#"
7,300,Angels,Robbie Williams,,0,"E,E,A,C#m,B,E,A,C#m,B,A,C#m,A,D,A,E,D,A,E,B,C#..."
8,407,The Difference,The Wallflowers,,0,"E,A,E,A,B,E,C#,B,E,A,E,A,B,E,C#,B,E,A,E,A,B,E,..."
9,464,I Lay My Love On You,Westlife,,0,"Am,Dm,G,C,Em,Am,Dm,F,G,C,G,Am,Dm,G,C,Em,Am,Dm,..."


In [106]:
def fix_accidental(note, accidental):
    notes = np.asarray(['A', 'B', 'C', 'D', 'E', 'F', 'G'])

    note_idx = int(np.where(notes == note)[0])
    if accidental == '#':
        # Account for B# or E#
        if note == 'B':
            return('C', '')
        elif note == 'E':
            return('F', '')
        return(note,accidental)
    elif accidental == 'b':
        # Account for Cb or Fb
        if note == 'C':
            return('B', '')
        elif note == 'F':
            return('E', '')
        return(notes[note_idx - 1],'#')
    else:
        return(note, accidental)
            
        
def clean_chords(chords):
    """
    This function takes in a comma-separated string of 
    chords and cleans it by removing any base note variations, or
    other chord embelishments.  Diminished labels are kept as these
    are used in the chord progression table. The purpose of this
    is to clean the chords to match the labels within the chord
    progression table.
    
    returns:
        new_chords - array of newly cleaned chords to be tabulated
                        by the chord progression table
    """
    
    # Pattern grouping: 1=(chord pitch) 2=(base note) 3=(chord type) 4=(base note)
    pattern = "^([A-G]+)(\/[A-G]*[b#])*([(?m)|(?m\d)|(?b\d)|(?#\d)|(?maj\d)|\
    (?add\d)|(?sus\d)|(?aug)|(?aug\d)|(?dim)|(?dim\d)]*)(\/[A-G]*[b#])*"        
    prog = re.compile(pattern)

    pattern2 = "^([A-G])([b#])?(m$|m\d$)?(dim$|dim\d$)?"
    prog2 = re.compile(pattern2)

    chords = chords.split(',')
    new_chords = [""]*len(chords)
    for i in range(len(chords)):
        curr_chord = chords[i]
        groups = prog.findall(curr_chord)[0] 
        no_base = groups[0] + groups[2]
        no_num = re.sub(pattern="\d", repl="", string=no_base)

        groups = prog2.findall(no_num)[0]
        note,accidental = fix_accidental(groups[0], groups[1])
        new_chords[i] = note + accidental + groups[2] + groups[3]
        
    return(new_chords)


def get_key_tbls():
    """
    Helper function that reads in a chord progression table
    and creates a dictionary that maps chord names to their
    indices on the chord progression table. This dictionary
    will be used to tabulate a 'key table' to determine the
    key of a song.
    
    returns:
        Key_dict - Dictionary mapping chord names to indices
        Keys - array of keys that correspond to the order
                of the progression table
    """
    Key_tbl = pd.read_csv('../data/key_table.csv')
    Keys = list(Key_tbl.key)
    Tbl = np.asmatrix(Key_tbl.iloc[:,1:8])

    # Storing all possible chords 
    all_chords = []
    for i in range(Tbl.shape[0]):
        for j in np.asarray(Tbl[i])[0]:
            all_chords.append(j)
    all_chords = np.unique(all_chords)
    
    # Creating dict(key='chord', val='indices in progression tbl')
    Key_dict = {}
    for chord in all_chords:
        Key_dict[chord] = np.where(Tbl == chord)

    return(Key_dict, Keys)

def is_rel_min(comp_key, act_key):
    """
    Determines if the actual key marked indicated
    UltimateGuitarTabs.com is the relative minor
    of the actual key (i.e., Am(rel. minor) and C(actual))
    
    args:
        comp_key - Computed key
        act_key - UltimateGuitarTabs.com key
    returns:
        - True if it's relative minor, False otherwise
    """
    notes = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']
    if str(act_key[-1]) != 'm': #If not minor, return 
        return(False)
    if len(comp_key) == 2:
        note, accidental = fix_accidental(comp_key[0],comp_key[1])
        comp_key = note + accidental
    comp_idx = np.where(np.asarray(notes) == comp_key)[0][0]
    rel_min_idx = comp_idx - 3
    if notes[rel_min_idx] + 'm' == act_key:
        return(True)

def compute_key(Key_dict, Keys, chords):
    """
    Computes the key of a song by analyzing
    the chords used within the song. A theoretical
    key matrix where each row specifies the types of chords
    that coincide with a key is used to tabulate
    the existing chords. The largest row sum is the
    computed key of the song. 
    
    args:
        Key_dict - Python dictionary containing all possible
                   chords and their indexes in the theoretical key matrix.
                   This is used to tabulate a matrix of zeros.
        Keys - List of possible keys corresponding to theoretical key matrix
        chords - String of chords separated by commas
                   
    returns:
        - The computed key
        - List of cleaned chords, only major or minor
    """
    chords = clean_chords(chords)
    count_mat = np.zeros((12,7)) # Matrix of zeros to tabulate chord occurences
    start_chord = chords[0]
    # Tabulating chords
    for chord in chords:
        count_mat[Key_dict[chord]] += 1
        
    computed_key = Keys[np.argmax(np.sum(count_mat, axis = 1))]
    return(computed_key, chords)
    
def transpose_C(Key_dict, Keys, chords, states, song_id):
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    comp_key, chords = compute_key(Key_dict, Keys, chords)
    
    new_chords = [""]*len(chords) # Empty list for new chords
    if comp_key == 'C':
        for chord in chords:
            states.add(chord)
        return(states, \
               {'song_id' : song_id, \
                'orig_key': comp_key, \
                'trans_chords': chords})
    else:
        steps = np.where(np.asarray(notes) == comp_key)[0][0]
        for i in range(len(chords)):
            is_min = False
            is_dim = False
            curr_chord = chords[i]
            if curr_chord[-1] == 'm':
                if curr_chord[-3:len(curr_chord)] == 'dim':
                    is_dim = True
                    curr_chord = curr_chord[0:-3]
                else:
                    is_min = True
                    curr_chord = curr_chord[0:-1]
            
            chord_idx = np.where(np.asarray(notes) == curr_chord)[0][0]
            new_chord_idx = chord_idx - steps
            if is_min:
                new_chords[i] = notes[new_chord_idx] + 'm'
            elif is_dim:
                new_chords[i] = notes[new_chord_idx] + 'dim'
            else:
                new_chords[i] = notes[new_chord_idx]
            states.add(new_chords[i])
        return(states, \
               {'song_id' : song_id, \
                'orig_key': comp_key, \
                'trans_chords': new_chords})
def get_songs():
    Key_dict, Keys = get_key_tbls()

    songs = [""]*Chords.shape[0]
    states = set()
    for i in range(Chords.shape[0]):
        states, songs[i] = transpose_C(Key_dict, Keys, Chords['Chords'][i], states, Chords['id'][i])
    states = sorted(list(states))
    return(states, songs)

def write_clean_songs(songs):
    db = connect_to_database()
    cur = db.cursor()
    
    delete = "DROP TABLE IF EXISTS Clean_Chords;"
    cur.execute(delete)
    
    create = "CREATE TABLE Clean_Chords (id INT(11) NOT NULL PRIMARY KEY, orig_key TEXT(5), chords TEXT(500));"
    cur.execute(create)
    
    for i in range(len(songs)):
        
        song = songs[i]
        song_id = int(song['song_id'])
        orig_key = song['orig_key']
        chords = ','.join(song['trans_chords'])
        
        sql_tab = "INSERT INTO Clean_Chords (id,orig_key,chords) \
        VALUES ('%d','%s','%s')" % \
        (song_id, orig_key, chords) 
        
        try:
            cur.execute(sql_tab)
            db.commit()
        except:
            db.rollback()

def write_states(states):
    db = connect_to_database()
    cur = db.cursor()
    delete = "DROP TABLE IF EXISTS States;"
    cur.execute(delete)
    
    create = "CREATE TABLE States (states TEXT(500));"
    cur.execute(create_tabs)
    
    sql_tab = "INSERT INTO States (states) VALUES ('%s')" % \
        ','.join(states)
    try:
        cur.execute(sql_tab)
        db.commit()
    except:
        db.rollback()

def create_transition_mat(states, chords):
    num_states = len(states)
    trans_mat = np.zeros((num_states, num_states))
    for i in range(1, len(chords)):
        curr_chord = chords[i - 1]
        curr_chord_idx = np.where(np.asarray(states) == curr_chord)[0][0]
        
        next_chord = chords[i]
        next_chord_idx = np.where(np.asarray(states) == next_chord)[0][0]
        trans_mat[curr_chord_idx, next_chord_idx] += 1

    row_sums = np.sum(trans_mat, axis = 1)
    trans_mat = trans_mat/row_sums
    return(np.nan_to_num(trans_mat))

def compute_euclidean(mat1, mat2):
    # Computes euclidean distance between two
    # transition matrices
    # NOTE: Will produce runtime warning with division by zero
    mat_dif = mat1 - mat2
    return(math.sqrt(np.sum(np.multiply(mat_dif, mat_dif)))/mat1.shape[0])

def get_similar_songs(states, song_id, clean_chords):
    curr_song = clean_chords.loc[clean_chords['id'] == song_id]
    chords = curr_song.iat[0, 2].split(',')#[0,2] As it's just one song and chords at idx 2
    tm = create_transition_mat(states,chords)
    dist = [0]*clean_chords.shape[0]
    for i in range(len(dist)):
        curr_tm = create_transition_mat(states,clean_chords['chords'][i].split(','))
        dist[i] = compute_euclidean(tm, curr_tm)
        
    dist_ord = np.argsort(dist) # Sorting by distance ascending
    
    idxs = np.unique(list(Chords['Song'][dist_ord]), return_index=True)[1] # Indexes of unique entries
    # np.unique() automatically sorts, so we must resort by distance
    sim_songs = Chords.iloc[dist_ord[sorted(idxs)]]
    return(sim_songs)


In [29]:
# Writing states and clean songs tables
states, songs = get_songs()
write_clean_songs(songs)
write_states(states)

    id         Song Artist Key  Capo  \
4  118  Los Angeles      X         0   

                                              Chords  
4  E,C,D,G,A,A,G,D,C,A,G,D,C,A,G,C,A,G,C,A,G,C,A,...  


In [63]:
Clean_Chords = pd.DataFrame(list(get_table('Clean_Chords')), columns=['id','orig_key', 'chords'])
states = get_table('States')[0][0].split(',')
states
Clean_Chords.loc[Clean_Chords['id'] == 118]

,id,orig_key,chords
4,118,G,"A,F,G,C,D,D,C,G,F,D,C,G,F,D,C,F,D,C,F,D,C,F,D,..."


In [109]:
sim = get_similar_songs(states, 118, Clean_Chords)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:240: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:240: RuntimeWarning: divide by zero encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:248: RuntimeWarning: overflow encountered in multiply


In [121]:
sim

,id,Song,Artist,Key,Capo,Chords
4,118,Los Angeles,X,,0,"E,C,D,G,A,A,G,D,C,A,G,D,C,A,G,C,A,G,C,A,G,C,A,..."
480,698704,Remembering Sunday,All Time Low,A,2,"Cadd9,Em,Cadd9,A7,Cadd9,Em,Cadd9,A7,Cadd9,D,Em..."
17,4019,You Cant Always Get What You Want,The Rolling Stones,,0,"G,Gadd9,C,A,D,G,Gadd9,G,C,G,Gadd9,G,C,G,Gadd9,..."
762,1189512,22,Taylor Swift,,0,"G,Dsus4,Cadd9,Dsus4,G,Dsus4,Cadd9,Dsus4,G,Dsus..."
501,746376,Toes,Zac Brown Band,,2,"A,D,A,E,A,D,A,E,A,A,D,A,E,D,E,A,A,D,A,E,A,D,A,..."
109,44578,Mother,Pink Floyd,,0,"G,Cadd9,G,G,Cadd9,G,Cadd9,G,D,Cadd9,Cadd9,G,G,..."
116,47489,Where Did You Sleep Last Night,Nirvana,E,0,"E,A,G,B,E,E,A,G,B,E,E,A,G,B,E,E,A,G,B,E,E,A,G,..."
307,277200,Joy To The World,Misc Christmas,G,0,"G,D,G,C,D,G,G,G,G,G,G,D,D,G,C,G,D,G,G,D,G,C,D,..."
517,760569,You Found Me,The Fray,,0,"E,B,G#m,F#,E,B,G#m,F#,B,E,E,B,G#m,F#,B,E,E,B,G..."
441,621851,You Shook Me All Night Long,AC/DC,,0,"G,C,D,G,C,D,G,G,C,D,G,G,C,D,D7,G,C,D,C,G,C,D,C..."
